# Building a Neural Net From Scratch to Predict MNIST Handwritten Digits

The Imports

In [1]:
import pandas as pd
import random
import math

Import the data

In [2]:
def importData(sources):
    data = []

    for source in sources:
        dat = pd.read_csv(source)
        data += dat.values.tolist()

    return data

def trainValTestSplit(data, trainSize, valSize):

    train = data[:int(trainSize*len(data))]
    val = data[int(trainSize*len(data)):int((trainSize+valSize)*len(data))]
    test = data[int((trainSize+valSize)*len(data)):]

    return train, val, test


In [3]:
class Connection:
    def __init__(self, previousPerceptron, nextPerceptron, weight = 1):
        self.weight = weight
        self.previousPerceptron = previousPerceptron
        self.nextPerceptron = nextPerceptron

    def updateWeight(self, delta):
        self.weight += delta

    def getWeight(self):
        return self.weight

    def getPreviousPerceptron(self):
        return self.previousPerceptron.getOutput()

In [4]:
class Perceptron:

    #Create the perceptron, only takes in it's bias
    def __init__(self, bias = 1):
        self.bias = bias
        self.backwardsConnections = []
        self.forwardsConnections = []
        self.output = 0


    #This method is used to add a forward connection to the perceptron
    def addForwardsConnection(self, connection):
        self.forwardsConnections.append(connection)


    #This takes in the layer before it and creates connections to all of the perceptrons in that layer
    # The connections are made of connection objects which just contain the node before it, the node after it, and the weight
    # The previous layer is a list of perceptrons
    def makeConnections(self, previousLayer):
        for perceptron in previousLayer:
            connection = Connection(perceptron, self)
            perceptron.addForwardsConnection(connection)

            self.backwardsConnections.append(connection)


    #This method returns the output of the perceptron
    def getOutput(self):
        return self.output


    #The activation function is a sigmoid function
    def activationFunction(self, value):
        return 1/(1+math.exp(-value))


    #This is the function that is called to calculate the output of the perceptron
    #Loops through all the back connections it has and gets the output of the previous perceptron and multiplies it by the weight
    #Then adds the bias and uses the activation function
    #Stores the output in the output variable
    def findOutput(self):
        value = 0

        #The sum of all the weights times the output of the previous perceptron
        for connection in self.backwardsConnections:
            value += connection.getWeight() * connection.previousPerceptron.getOutput()

        #Add the bias
        value += self.bias

        #Use the activation function
        self.output = self.activationFunction(value)


    #Updates the bias of the perceptron
    def updateBias(self, delta):
        self.bias += delta

    #For the input layer it needs to be given the first value
    def giveOutput(self, value):
        self.output = value

    def __repr(self):
        output = "Bias: " + str(self.bias) \
            + " Output: " + str(self.output) \
            + " Backwards Connections: " \
            + str(self.backwardsConnections) \
            + " Forwards Connections: " \
            + str(self.forwardsConnections)
        return output


    def __str__(self):
        output = "Bias: " + str(self.bias) \
            + " Output: " + str(self.output) \
            + " Backwards Connections: " \
            + str(self.backwardsConnections) \
            + " Forwards Connections: " \
            + str(self.forwardsConnections)
        return output


Going to make a 1 by 1 by 1 NN for testing

In [5]:
layer1 = [Perceptron(), Perceptron(), Perceptron()]
layer2 = [Perceptron(), Perceptron(), Perceptron(), Perceptron()]
layer3 = [Perceptron(), Perceptron()]

NN = [layer1, layer2, layer3]

# print("First Perceptron")
# print(layer1[0])

for perceptron in layer2:
    perceptron.makeConnections(layer1)

# set connections for the third layer
for perceptron in layer3:
    perceptron.makeConnections(layer2)

# print("first Perceptron after connections")
# print(layer1[0])

# print("second Perceptron after connections")
# print(layer2[0])

# print("third Perceptron after connections")
# print(layer3[0])

In [6]:


# set connections for the second layer

#Input will be random for now
input = [int(random.random()*10) for i in range(len(layer1))]

print("input: " + str(input))


for i in range(len(NN)):
    if i == 0:
        if len(input) != len(NN[i]):
            print("Error: Input size does not match input layer size")
            break
        else:
            for j in range(len(NN[i])):
                NN[i][j].giveOutput(input[j])

    else:
        for perceptron in NN[i]:
            perceptron.findOutput()


print("Output:")

for i in range(len(layer3)):
    print(layer3[i].getOutput())




input: [6, 7, 3]
Output:
0.9933071479748135
0.9933071479748135


## Build the Neural Network

In [40]:
class NeuralNetwork:
    def __init__(self, shape):
        self.layers = []

        #Build the network
        for i in range(len(shape)):
            layer = []

            for i in range(shape[i]):
                layer.append(Perceptron())

            self.layers.append(layer)


        #Connect the layers
        for i in range(1, len(self.layers)):
            for perceptron in self.layers[i]:
                perceptron.makeConnections(self.layers[i-1])


    def feedForward(self, input):
        if len(input) != len(self.layers[0]):
            print("Error: Input size does not match input layer size")
            return

        for i in range(len(self.layers)):
            if i == 0:
                for j in range(len(self.layers[i])):
                    self.layers[i][j].giveOutput(input[j])

            else:
                for perceptron in self.layers[i]:
                    perceptron.findOutput()

        output = []

        for perceptron in self.layers[-1]:
            output.append(perceptron.getOutput())


    def getOutput(self):
        output = []

        for perceptron in self.layers[-1]:
            output.append(perceptron.getOutput())

        return output


    def costFunction(self, expected, actual):
        cost = 0

        for i in range(len(expected)):
            cost += (expected[i] - actual[i])**2

        return cost


    def backPropogation(self, expected, actual):
        for i in range(len(expected)):
            delta = (expected[i] - actual[i]) * actual[i] * (1-actual[i])

            for connection in self.layers[-1][i].backwardsConnections:
                connection.updateWeight(delta)

            self.layers[-1][i].updateBias(delta)

        for i in range(len(self.layers)-2, 0, -1):
            for j in range(len(self.layers[i])):
                delta = 0

                for connection in self.layers[i][j].forwardsConnections:
                    delta += connection.getWeight() * connection.nextPerceptron.getOutput() * (1 - connection.nextPerceptron.getOutput())

                for connection in self.layers[i][j].backwardsConnections:
                    connection.updateWeight(delta)

                self.layers[i][j].updateBias(delta)


    def getPrediction(self, input):
        output = []

        self.feedForward(input)

        for perceptron in self.layers[-1]:
            ans = perceptron.getOutput()
            output.append(ans)

        return output


testing area

In [46]:
nn = NeuralNetwork([3, 4, 2])

for i in range(1000):

    nn.feedForward([1, 2, 3])

    # print("Prediciton = ", nn.getOutput())

    ans = [1, 0]

    # print("Cost = ")
    # print(nn.costFunction(ans, nn.getOutput()))

    nn.backPropogation(ans, nn.getOutput())


print("FINAL Prediciton = ", nn.getPrediction([1 ,2 ,3]))



FINAL Prediciton =  [0.9927154900640066, 0.007285078451897764]
